In [19]:
from bs4 import BeautifulSoup
import argparse
import re
import codecs
import time
import pandas as pd
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import threading
from urllib import request, parse
import urllib

In [20]:
def get_page_url(zipcode, page_num):
    return 'https://www.yelp.com/search?find_loc={0}+ny&start={1}'.format(zipcode, page_num)

# Return false as flag, indicating the url is bad
# or return soup
def get_soup(page_url):
#     params = { 'key':'f3fabf585b34bb1b871e723861473152',
#            'url': page_url}
#     encoded = parse.urlencode(params)
# ('http://api.scraperapi.com/' + '?' + encoded)
    headers = {'Accept': 'application/json'}
    myKey = 'f3fabf585b34bb1b871e723861473152' # get it after you register Scraper
    quote = parse.quote(page_url)
    
    counter = 0
    while True:
        try:
            req = request.Request('http://api.scraperapi.com/?key=' + myKey + '&url=' + quote)
            response_body = request.urlopen(req).read()
#         except RemoteDisconnected:
#             continue
        except:
            counter += 1
            if counter >3:
                return False # Stands for failed to open, just pass opening the link
            else:
                continue   
        break
        
    return BeautifulSoup(response_body)

In [21]:
# When the Yelp shows you "We're sorry, the page of results you requested is unavailable."
# Or 'suggestions for improving'
# Return False
# Otherwise the page contains shops' information, which is needed to be crawled.
# Check three times
def is_page_exist(url):
    idx = 0
    while idx<3:
        soup = get_soup(url)
        try:
            string1 = soup.find('h3', class_ = 'lemon--h3__373c0__5Q5tF heading--h3__373c0__1n4Of')
            string2 = str(soup.find('p', {'class': 'lemon--p__373c0__1hkz1 text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa'})).lower()
        
            if string1 == None and string2 == None:
                idx += 1
                time.sleep(5)
            elif 'We\'re sorry' in string1 or 'suggestions for improving' in string2:
                return 'NE'# Stands for not exist
        except:
            return 'pass'
            
    return 'pass'# Stands for failed to open, just pass opening the link

In [22]:
# soup = get_soup('https://www.yelp.com/search?find_loc=10006+ny&start=118')
# try:
#     restaurantList = soup.findAll('div', attrs={'class':re.compile(r'^search-result natural-search-result')})
#     if restaurantList != []:
#         return restaurantList, 'natural'
#     else:
#         restaurantList = soup.findAll('div', attrs={'class':'lemon--div__373c0__6Tkil searchResult__373c0__1yggB border-color--default__373c0__2oFDT'})
#         if restaurantList != []:
#             # meaning the element type is 'lemon'
#             return restaurantList, 'lemon'
#         else:
#             return is_page_exist('https://www.yelp.com/search?find_loc=10006+ny&start=118'), ''       
# except:
#     return 'pass', ''

In [23]:
def get_restaurant_list(page_url):    
    restaurantList = []
    soup = get_soup(page_url)
    
    # get the list of restaurants and check the element tag type
    try:
        restaurantList = soup.findAll('div', attrs={'class':re.compile(r'^search-result natural-search-result')})
        if restaurantList != []:
            return restaurantList, 'natural'
        else:
            restaurantList = soup.findAll('div', attrs={'class':'lemon--div__373c0__6Tkil searchResult__373c0__1yggB border-color--default__373c0__2oFDT'})
            if restaurantList != []:
                # meaning the element type is 'lemon'
                return restaurantList, 'lemon'
            else:
                return is_page_exist(page_url), ''       
    except:
        return 'pass', '' # Stands for failed to open, just pass opening the link

In [24]:
def create_temp_table(rankList = '', titleList = '', linkList = '', ratingList = '', reviewsList = '', yelpPageList = '', categoriesList = ''):
    temp = pd.DataFrame(columns = ['Rank', 'Title', 'Rating', 'Reviews', 'Categories', 'Link'])
    temp['Rank'] = rankList
    temp['Title'] = titleList
    temp['Link'] = linkList
    temp['Rating'] = ratingList
    temp['Reviews'] = reviewsList
    temp['Categories'] = categoriesList
    return temp

# The following functions accept tag as a indicator to check what kind of element name is used
def get_rank(r, tag):
    if tag == 'lemon':
        return re.findall(r'\d+', r.find('h3', {'class':'lemon--h3__373c0__5Q5tF heading--h3__373c0__1n4Of alternate__373c0__1uacp'}).text)[0]
    elif tag == 'natural':
        return re.findall(r'\d+', r.find(('h3', {'class':'search-result-title'})).text)[0]

def get_link(r, tag):
    if tag == 'lemon':
        return 'https://www.yelp.com' + str(r.find('a', {'class': 'lemon--a__373c0__1_OnJ link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5'}).attrs['href'])
    elif tag == 'natural':
        return 'need HELP!!!!!!!!!!!!'
    
def get_title(r, tag):
    if tag == 'lemon':
        return r.find('a', {'class':'lemon--a__373c0__1_OnJ link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5'}).getText()
    elif tag == 'natural':
        return r.find('a', {'class':'biz-name'}).getText()

def get_rating(r, tag):
    if tag == 'lemon':
        return r.find('span', attrs = {'class': 'lemon--span__373c0__1xR0D display--inline__373c0__1DbOG border-color--default__373c0__2oFDT'}).find('div').attrs['aria-label'].split()[0]
    elif tag == 'natural':
        return r.find('div', attrs = {'class': re.compile(r'^i-stars i-stars')})['title'][:3]

def get_reviews(r, tag):
    if tag == 'lemon':
        return re.sub(r'\D',"", r.find('span', class_ = 'lemon--span__373c0__1xR0D text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_').text.replace('\n',''))
    elif tag == 'natural':
        return re.sub(r'\D',"",r.find('span', class_ = re.compile(r'^review-count')).text.replace('\n',''))

def get_shop_link(r, tag):
    if tag == 'lemon':
        return r'https://www.yelp.com' + r.find('a', {'class':'lemon--a__373c0__1_OnJ link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5'})['href']
    elif tag == 'natural':
        return  r'https://www.yelp.com' + r.find('a', {'class':'biz-name'})['href']

def get_category(r, tag):
    if tag == 'lemon':
        categories = r.findAll('a', {'class':'lemon--a__373c0__1_OnJ link__373c0__29943 link-color--inherit__373c0__15ymx link-size--default__373c0__1skgq'})
        return ', '.join([i.getText() for i in categories])
        categories = r.find('span', {'class':'category-str-list'}).getText().replace('\n','').split(',')
        return ','.join([re.sub(r'^\s*', '', c) for c in categories])

In [25]:
def crawl_page(restaurantList, tag):
    rankList = []
    titleList = []
    linkList = []
    ratingList = []
    reviewsList = []
    yelpPageList = []
    categoriesList = []
    for r in restaurantList:        
        try:
            rankList.append(get_rank(r, tag)) 
        except Exception as e:
            print ('rank extract fail', str(e))

        try:
            titleList.append(get_title(r, tag))
        except Exception as e:
            print ('title extract fail', str(e))
        
        try:
            linkList.append(get_link(r, tag))
        except Exception as e:
            print ('link extract fail', str(e))
            
        try:
            ratingList.append(get_rating(r, tag))
        except Exception as e:
            print ('rating extract fail', str(e))

        try:
            reviewsList.append(get_reviews(r, tag))
        except Exception as e:
            print ('rating extract fail', str(e))

        try:
            yelpPageList.append(get_shop_link(r, tag))
        except Exception as e:
            print ('yelp page link extraction fail', str(e))

        try:
            categoriesList.append(get_category(r, tag))
        except Exception as e:
            print ("category extract fail", str(e))
            
    temp = create_temp_table(rankList, titleList, linkList, ratingList, reviewsList, yelpPageList, categoriesList)
    return temp

In [26]:
def create_file_name(zipcode, page_idx):
    fileName = str(zipcode) + '_' + str(page_idx) + '.csv'#  + ' to ' +  str(page_idx + 300)
    return fileName

def save_to_file(fileName, df):
    path = 'E:\\Project\\Molitor\\Yelp\\'
    fullPath = path + fileName
    df.to_csv(fullPath)
    print('Saved file: '+ fileName)
    
def get_zip_list():
    # change the file address based on your path
    filePath = "E:\\Project\\Molitor\\Yelp\\Copy of ZIP NYC 688.xlsx"
    reader = pd.read_excel(filePath)
    reader = reader.drop(['neighborhood', 'borough'], axis = 1)
    return reader['zip code'].tolist()

In [27]:
def crawler(zipcode, page_idx):
    fileName = create_file_name(zipcode, page_idx)
    df = create_temp_table()
    idx = 0
#     bad_link = []
    listLen = 0
    preSetIncrement = 30 ##10
    while idx < preSetIncrement:
        page_url = get_page_url(zipcode, page_idx)
        restaurantList, tag = get_restaurant_list(page_url)
        # check the availability of website
        if restaurantList == 'NE':
            print ('***', zipcode, ':', page_idx, 'Not Exist')
            idx = preSetIncrement+1
            page_idx = 1000
        elif restaurantList == 'pass':
#             bad_link.append(page_url)
            page_idx += listLen# default setting, previous page's length
            idx += 1
            print('can not open the page: ' + page_url)
            pass
        elif restaurantList != []: # page exists
            df = df.append(crawl_page(restaurantList, tag))
            listLen = len(restaurantList)
            page_idx += listLen
            idx += 1
        else:
            print('nothing meet the rules here')
            print(restaurantList)
            print(tag)
            print(page_url)
            idx+=1
            
    if len(df) != 0:
        save_to_file(fileName, df)
    return page_idx

In [ ]:
zipList = get_zip_list()
for zi in zipList[5:]:
    page_idx = 0
    while page_idx < 1000:
        page_idx = crawler(zi, page_idx)


*** 10006 : 116 Not Exist
Saved file: 10006_0.csv
Saved file: 10007_0.csv
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=320
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=330
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=340
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=350
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=360
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=370
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=380
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=390
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=400
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=410
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=420
can not open the page: https://www.yelp.com/search?find_l

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?find_loc=10007+ny&start=600
can not open the page: https://www.yelp.com/search?f

In [4]:
#         downloadThread = threading.Thread(target=crawler, args = (zi, page_idx))
#         downloadThread.start()
#         print ("One thread starts! It\'s for " + str(zi) + ': ' + str(page_idx) + '\n')#"The No."+str(idx+1)+
#         downloadThreads.append(downloadThread)
#         time.sleep(5)
    
# def foo(bar, baz):
#   print 'hello {0}'.format(bar)
#   return 'foo' + baz

# from multiprocessing.pool import ThreadPool
# pool = ThreadPool(processes=1)

# async_result = pool.apply_async(foo, ('world', 'foo')) # tuple of args for foo

# # do some other stuff in the main process

# return_val = async_result.get()  # get the return value from your function.

NameError: name 'zipList' is not defined

In [ ]:
# zipList = get_zip_list()
# downloadThreads = []
# for zi in zipList[:1]:
#     page_idx = 0
#     while page_idx < 1000:
# #         crawler(zi, page_idx)
#         downloadThread = threading.Thread(target=crawler, args = (zi, page_idx))
#         downloadThread.start()
#         print ("One thread starts! It\'s for " + str(zi) + ': ' + str(page_idx) + '\n')#"The No."+str(idx+1)+
#         downloadThreads.append(downloadThread)
#         time.sleep(5)
#         page_idx += 30
#     zi += 1

# # [t.join() for t in downloadThreads]

In [ ]:
# [t.join() for t in downloadThreads]